In [9]:

# installing the required module(s)

!pip3 install pinecone-client
!pip3 install -U python-dotenv


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Initializing pinecone with the api key and environment.

In [10]:
# importing required libraries

import pinecone
import os
from dotenv import load_dotenv

load_dotenv()

# initializing pinecone
pinecone.init(api_key=os.getenv("PINECONE_API_KEY"), environment="asia-southeast1-gcp-free")

Adding an example index in the db

In [14]:
# pinecone.create_index("example-index", dimension=128, pods=1)
# pinecone.describe_index("example-index")

IndexDescription(name='example-index', metric='cosine', replicas=1, dimension=128.0, shards=1, pods=1, pod_type='p1', status={'ready': True, 'state': 'Ready'}, metadata_config=None, source_collection='')

## Testing out the upserting and querying functions.

Getting random samples from the population being numbers between 0 and 128.

Upserting three to the example-index, and then querying with a fourth to see the output.

In [21]:
index = pinecone.Index(index_name="example-index")

# creating random vectors
import random
population = [x for x in range(128)]

# adding three random vectors sampled from the population above
for i in range(3):
    random_sample = random.sample(population, k=128)
    index.upsert([(str(i), random_sample)])

In [ ]:
query_sample = random.sample(population, k=128)
print(f"Querying with the vector: {query_sample}")

index.query(
    vector=query_sample,
    top_k=2,
    include_values=True
)

In [49]:
# deleting the example index
pinecone.delete_index("example-index")

## Testing out basic embeddings, upserting and querying embedded sentences

In [44]:
# required modules for openai.embeddings_utils

!pip3 install matplotlib
!pip3 install plotly
!pip3 install scipy
!pip3 install scikit-learn


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 8.3/8.3 MB 15.1 MB/s eta 0:00:00
     ------------------------------------- 298.0/298.0 kB 19.2 MB/s eta 0:00:00
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [80]:
# initializations

import openai
import pandas as pd

openai.api_key = os.getenv("OPENAI_API_KEY")

# creating the new index
EMBEDDING_DIMENSIONS = 1536 # found on https://openai.com/blog/new-and-improved-embedding-model
pinecone.create_index("sentences-index", dimension=EMBEDDING_DIMENSIONS, pods=1)
pinecone.describe_index("sentences-index")


IndexDescription(name='sentences-index', metric='cosine', replicas=1, dimension=1536.0, shards=1, pods=1, pod_type='p1', status={'ready': True, 'state': 'Ready'}, metadata_config=None, source_collection='')

Adding sentences from a paragraph to the vector database.

In [82]:

import time

paragraph = """This new world is called Talis. 
It has six regions, all ruled by descendants of the first king. 
Valterre is the largest region, stretching across a massive continent. 
Dehjet is an isle nation in the center of the world, surrounded by a dense and poisonous fog. 
It was cursed after the god of magic, Khiosa, punished the first king of Dehjet for using magic to bring the dead back to life. 
The first king was the creator of necromancy. No magic can be used within the fog, and the fog kills anything alive that walks through.
Necromancy as a practice is outlawed in all six regions, but it is not the only type of magic that exists in Talis. """

sentence_list = paragraph.split('\n')
index = pinecone.Index(index_name="sentences-index")

for i, sentence in enumerate(sentence_list):
    embedding = openai.Embedding.create(input = [sentence], model='text-embedding-ada-002')['data'][0]['embedding']
    index.upsert([(sentence, embedding)])

    time.sleep(1)

Querying the vector database.

In [88]:
user_sentence = "Tell me about the fog around Dehjet"
embedding = openai.Embedding.create(input = [user_sentence], model='text-embedding-ada-002')['data'][0]['embedding']

query_response = index.query(
    vector=embedding,
    top_k = 3,
    include_values=True
)

query_response['matches'][-1]['id']

'The first king was the creator of necromancy. No magic can be used within the fog, and the fog kills anything alive that walks through.'